Run this notebook in the same virtual environment with superlinked server
to ensure the same version of `superlinked` framework.

In [1]:
!pip freeze | grep superlinked

superlinked==19.14.1


In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import sys

cwd = Path.cwd()
project_dir = cwd.parent
sys.path.append(str(project_dir))

In [4]:
from superlinked import framework as sl

from superlinked_app.index import index, hotel_schema
from superlinked_app.query import query
from superlinked_app.api import vector_database

/Users/benjiqu/Projects/superlinked/hotel-search/.venv-new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
source = sl.InMemorySource(hotel_schema)
executor = sl.InteractiveExecutor(
    sources=[source],
    indices=[index],
    vector_database=vector_database,
)
app = executor.run()

In [6]:
params = {
    "natural_query": "well rated pet friendly hotel in london, cozy interior",
    "limit": 3,
}
result = app.query(query, **params)
print(result.metadata.search_params)
sl.PandasConverter.to_pandas(result)


{'price_weight': 0.0, 'rating_weight': 1.0, 'rating_count_weight': 0.0, 'description_weight': 1.0, 'description': 'Pet-friendly hotel with cozy interior.', 'limit': 3, 'city': 'London', 'min_price': None, 'max_price': None, 'min_rating': None, 'max_rating': None, 'accomodation_types_include': 'Hotel', 'accomodation_types_exclude': None, 'property_amenities_include_all': None, 'property_amenities_include_any': ['Pets allowed'], 'property_amenities_exclude': None, 'room_amenities_include_all': None, 'room_amenities_include_any': None, 'room_amenities_exclude': None, 'wellness_spa_include_all': None, 'wellness_spa_include_any': None, 'wellness_spa_exclude': None, 'accessibility_include_all': None, 'accessibility_include_any': None, 'accessibility_exclude': None, 'for_children_include_all': None, 'for_children_include_any': None, 'for_children_exclude': None, 'natural_query': 'well rated pet friendly hotel in london, cozy interior', 'system_prompt_param__': "Extract the search parameters f

,description,price,rating,rating_count,accomodation_type,property_amenities,room_amenities,wellness_spa,accessibility,for_children,city,country,id,similarity_score
0,"Welcome to Canopy by Hilton London City, an im...",145,9.4,3119,Hotel,"[24-hour reception, 24-hour room service, Busi...","[Air conditioning, Bathtub, Cable TV, Central ...",[],"[Accessible bathroom, Accessible hotel, In-roo...",[Cot],London,United Kingdom,Canopy by Hilton London City,0.742976
1,Just a short walk from the Aldgate tube statio...,117,8.9,6329,Hotel,"[24-hour reception, Breakfast, Cashless paymen...","[Air conditioning, Central heating, Desk, Free...",[],[Wheelchair accessible],[],London,United Kingdom,Motel One London-Tower Hill,0.712467
2,Begin your day with a scrumptious on-site brea...,120,9.1,1867,Hotel,"[24-hour reception, Café, Cashless payment, Co...","[Air conditioning, Balcony, Central heating, F...",[],[Wheelchair accessible],[Cot],London,United Kingdom,"The Gantry London, Curio Collection By Hilton",0.704894


### Partial scores

In [7]:
from superlinked.framework.dsl.executor.query.query_executor import (
    QueryExecutor,
    QueryDescriptor,
    QueryParamValueSetter,
    QueryVectorFactory,
)
from superlinked.evaluation.vector_sampler import VectorCollection
from superlinked.framework.dsl.space.space import Space

import numpy as np
import pandas as pd

In [8]:
def get_query_vector(
    index: sl.Index,
    app: sl.InteractiveExecutor,
    query_descriptor: QueryDescriptor,
    params: dict,
):
    query_vector_factory = QueryVectorFactory(index._dag)
    query_executor = QueryExecutor(app, query_descriptor, query_vector_factory)
    query_descriptor = QueryParamValueSetter.set_values(query_descriptor, params)
    query_vector = query_executor._produce_query_vector(query_descriptor)
    return query_vector


def get_partial_scores(
    spaces: list[Space], stored_vectors: VectorCollection, query_vector: np.ndarray
) -> dict[str, dict[str, float]]:

    i_start = 0
    partial_scores = {}
    ids = stored_vectors.id_list

    for space in spaces:
        fields = list(space._field_set)
        assert len(fields) == 1
        name = fields[0].name
        i_end = i_start + space.length
        v_space = stored_vectors.vectors[:, i_start:i_end]
        q_space = query_vector[i_start:i_end]
        partial_scores_space = np.dot(v_space, q_space)

        partial_scores[name] = {id: x for id, x in zip(ids, partial_scores_space)}
        i_start = i_end

    return partial_scores

In [10]:
from superlinked.framework.dsl.executor.query.query_executor import (
    QueryExecutor
)   
from superlinked.framework.dsl.query.query_vector_factory import QueryVectorFactory

In [11]:
params = {
    "natural_query": "well rated pet friendly hotel in london, cozy interior",
    "limit": 3,
}

query_descriptor = QueryParamValueSetter.set_values(query, params)
query_vector_factory = QueryVectorFactory(index._dag)
query_executor = QueryExecutor(app, query_descriptor, query_vector_factory) 
knn_params = query_executor._map_search_params(query_descriptor)

print(knn_params)


{'price_weight': 0.0, 'rating_weight': 1.0, 'rating_count_weight': 0.0, 'description_weight': 1.0, 'description': 'Pet friendly hotel with cozy interior.', 'limit': 3, 'city': 'London', 'min_price': None, 'max_price': None, 'min_rating': None, 'max_rating': None, 'accomodation_types_include': 'Hotel', 'accomodation_types_exclude': None, 'property_amenities_include_all': None, 'property_amenities_include_any': ['Pets allowed'], 'property_amenities_exclude': None, 'room_amenities_include_all': None, 'room_amenities_include_any': None, 'room_amenities_exclude': None, 'wellness_spa_include_all': None, 'wellness_spa_include_any': None, 'wellness_spa_exclude': None, 'accessibility_include_all': None, 'accessibility_include_any': None, 'accessibility_exclude': None, 'for_children_include_all': None, 'for_children_include_any': None, 'for_children_exclude': None, 'natural_query': 'well rated pet friendly hotel in london, cozy interior', 'system_prompt_param__': "Extract the search parameters f

In [12]:
sl.PandasConverter.to_pandas(result)

,description,price,rating,rating_count,accomodation_type,property_amenities,room_amenities,wellness_spa,accessibility,for_children,city,country,id,similarity_score
0,"Welcome to Canopy by Hilton London City, an im...",145,9.4,3119,Hotel,"[24-hour reception, 24-hour room service, Busi...","[Air conditioning, Bathtub, Cable TV, Central ...",[],"[Accessible bathroom, Accessible hotel, In-roo...",[Cot],London,United Kingdom,Canopy by Hilton London City,0.742976
1,Just a short walk from the Aldgate tube statio...,117,8.9,6329,Hotel,"[24-hour reception, Breakfast, Cashless paymen...","[Air conditioning, Central heating, Desk, Free...",[],[Wheelchair accessible],[],London,United Kingdom,Motel One London-Tower Hill,0.712467
2,Begin your day with a scrumptious on-site brea...,120,9.1,1867,Hotel,"[24-hour reception, Café, Cashless payment, Co...","[Air conditioning, Balcony, Central heating, F...",[],[Wheelchair accessible],[Cot],London,United Kingdom,"The Gantry London, Curio Collection By Hilton",0.704894


In [13]:
result = app.query(query, **knn_params)

In [14]:
res = [e.metadata for e in result.entries]
for r in res:
    print(r)


score=0.74779844 partial_scores=[0.2500174350013831, 0.0, 0.49778098, 0.0]
score=0.71380776 partial_scores=[0.22125310191774006, 0.0, 0.49255466, 0.0]
score=0.70304483 partial_scores=[0.20803297460660336, 0.0, 0.49501184, 0.0]
